In [1]:
from vietadminunits import parse_address
import pandas as pd
import re
from unidecode import unidecode

In [2]:
def create_key(text):
    if not isinstance(text, str):
        return text
    u_text = unidecode(text) # Unidecode first help remove special characters
    c_text = u_text.lower().strip() # Case must be second step
    c_text = re.sub(r"\-|\'", ' ', c_text)
    c_text = c_text.replace(' ', '')
    return c_text

In [3]:
def find_new_district_english_module(text):
    au = parse_address(text, 2)
    return au.district_english

In [4]:
def prepare_address(province, change):
    address = province + ' ' + re.sub(f'.* thành ', '', change)
    return address

In [5]:
def create_province_english_module(text):
    au = parse_address(text, 1)
    return au.province_english

In [6]:
def create_district_key_module(text):
    au = parse_address(text)
    return au.district_key

In [7]:
def create_province_key_module(text):
    au = parse_address(text)
    return au.province_key

In [8]:
df = pd.read_csv('../data/input/danhmuchanhchinhgso.gov.vn.csv')

In [9]:
df['ward_address'] = df['long_ward'].fillna('') + ', ' + df['long_district'] + ', ' + df['long_province']
ward_addresses = df['ward_address'].dropna().tolist()

In [10]:
df

,long_province,long_district,long_ward,ward_level,ward_address
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường,"Phường Phúc Xá, Quận Ba Đình, Thành phố Hà Nội"
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường,"Phường Trúc Bạch, Quận Ba Đình, Thành phố Hà Nội"
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường,"Phường Vĩnh Phúc, Quận Ba Đình, Thành phố Hà Nội"
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường,"Phường Cống Vị, Quận Ba Đình, Thành phố Hà Nội"
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường,"Phường Liễu Giai, Quận Ba Đình, Thành phố Hà Nội"
...,...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã,"Xã Viên An Đông, Huyện Ngọc Hiển, Tỉnh Cà Mau"
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã,"Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau"
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn,"Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau"
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã,"Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau"


## Find wards that are the same keyword with provinces

In [35]:
def create_district_key(text):
    if not isinstance(text, str):
        return text
    u_text = unidecode(text) # Unidecode first help remove special characters
    c_text = str(u_text).lower().strip() # Case must be second step
    if not re.search(r'^Quan \d{1,2}', c_text, flags=re.IGNORECASE):
        c_text = re.sub(r'\sDistrict$|^Quan\s|^Huyen\s|^Thanh\sPho\s|\sCity$|^Thi\sXa\s|\sTown$|\s0', '', c_text, flags=re.IGNORECASE)
    c_text = re.sub(r"\-|\'", ' ', c_text)
    c_text = re.sub(r'\bqui\b', 'quy', c_text)
    c_text = c_text.replace(' ', '')
    return c_text

In [36]:
def create_province_key(text):
    u_text = unidecode(text) # Unidecode first help remove special characters
    c_text = str(u_text).lower().strip() # Case must be second step
    c_text = re.sub(r'\sProvince$|^Tinh\s|^Thanh\sPho\s|\sCity$', '', c_text, flags=re.IGNORECASE)
    c_text = re.sub(r"\-|\'", ' ', c_text)
    c_text = re.sub(r'\bqui\b', 'quy', c_text)
    c_text = c_text.replace(' ', '')
    return c_text

In [37]:
wrong_addresses = []
for row in df.itertuples():
    unit = parse_address(row.ward_address, level=1)
    
    data = {
        'ward_address': row.ward_address,
        'province': row.long_province,
        'wrong_province_key': unit.province_key,
    }
    
    if unit.province == None:
        print(row.ward_address)
        wrong_addresses.append(data)
        
    # elif unit.district == None:
    #     print(row.ward_address)
    #     wrong_addresses.append(data)
    
    elif unit.province_key != create_province_key(row.long_province):
        print(row.ward_address)
        wrong_addresses.append(data)

In [38]:
df_wrong_provinces = pd.DataFrame(wrong_addresses)

In [39]:
df_wrong_provinces

""


In [40]:
double_check_provinces = {}

if df_wrong_provinces.shape[0]:
    df_wrong_provinces['true_province_key'] = df_wrong_provinces.province.apply(create_province_key)
    
    
    for row in df_wrong_provinces.itertuples():
        double_check_provinces[row.wrong_province_key] = df_wrong_provinces[df_wrong_provinces['wrong_province_key']==row.wrong_province_key]['true_province_key'].tolist()

In [41]:
double_check_provinces

{}

## Find wards that are the same keyword with districts

We have to add `double_check_provinces` to module to solve province level before doing this step.

In [42]:
wrong_addresses = []
for row in df.itertuples():
    try:
        unit = parse_address(str(row.ward_address), level=2)
    except Exception as e:
        raise Exception(row.ward_address)
    
    # unit = parse_address(str(row.ward_address), level=3)
    
    data = {
        'ward_address': row.ward_address,
        'province': row.long_province,
        'district': row.long_district,
        'wrong_district_key': unit.district_key,
    }
    
    if unit.province == None:
        print(row.ward_address)
        wrong_addresses.append(data)
    elif unit.district == None:
        print(row.ward_address)
        wrong_addresses.append(data)
    # elif unit.ward == None and unit.district_key not in ['bachlongvi', 'conco', 'hoangsa', 'lyson', 'condao']:
    #     print(row.ward_address)
    #     wrong_addresses.append(data)
    
    elif unit.district_key != create_district_key(row.long_district):
        print(row.ward_address, unit.district_key, create_district_key(row.long_district))
        wrong_addresses.append(data)

In [44]:
df_wrong_districts = pd.DataFrame(wrong_addresses)

In [45]:
df_wrong_districts

""


In [46]:
double_check_districts = {}
if df_wrong_districts.shape[0]:
    df_wrong_districts = df_wrong_districts.drop(columns='ward_address').drop_duplicates()
    
    df_wrong_districts['district_address'] = df_wrong_districts['district'].apply(create_district_key) + ', ' + df_wrong_districts['province']
    
    df_wrong_districts['true_district_key'] = df_wrong_districts['district_address'].apply(create_district_key_module)
    
    for row in df_wrong_districts.itertuples():
        double_check_districts[row.wrong_district_key] = df_wrong_districts[df_wrong_districts['wrong_district_key']==row.wrong_district_key]['true_district_key'].tolist()

In [47]:
double_check_districts

{}

In [48]:
df_wrong_districts[df_wrong_districts.true_district_key==df_wrong_districts.wrong_district_key]

AttributeError: 'DataFrame' object has no attribute 'true_district_key'